In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import tensorflow as tf
from tensorflow.keras import optimizers, layers, models
# from tensorflow_addons.metrics import F1Score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tqdm import tqdm

from sklearn.model_selection import train_test_split
# from tensorflow.keras.layers import Input, TimeDistributed, Dense

In [2]:
CFG = {
    'VIDEO_LENGTH': 50,
    'IMG_SIZE': 128,
    'EPOCHS': 10,
    'LEARNING_RATE': 1e-4,
    'BATCH_SIZE':4,
    'SEED': 714
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(CFG['SEED']) ## Seed 고정

In [3]:
def frames_from_video_file(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    for _ in range(CFG['VIDEO_LENGTH']):
        _, img = cap.read()
        img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        img = img / 255
        frames.append(img)
    result = np.array(frames)[..., [2,1,0]]
    return result

class FrameGenerator:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __call__(self):
        for tp in self.dataframe.itertuples():
            video_frames = frames_from_video_file(tp.video_path)
            label = tp.label
            label = to_categorical(label, 13)

            yield video_frames, label

In [4]:
df = pd.read_csv('./train.csv')

train, val, _, _ =train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])
weights = compute_class_weight(class_weight='balanced', classes=np.unique(train.label), y=train.label)

In [5]:
## Model에 주입하기 위한 dataset 객체 생성

output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(13), dtype=tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(train),
                                          output_signature=output_signature)

val_ds = tf.data.Dataset.from_generator(FrameGenerator(val),
                                        output_signature=output_signature)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
train_ds = train_ds.batch(CFG['BATCH_SIZE'])
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.batch(CFG['BATCH_SIZE'])

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames:{train_frames.shape}')
print(f'Shape of training labels:{train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames:{val_frames.shape}')
print(f'Shape of validation labels:{val_labels.shape}')

2023-03-09 23:52:41.271678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-09 23:52:41.353845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sangho/.pyenv/versions/3.7.16/envs/python37/lib/python3.7/site-packages/cv2/../../lib64:
2023-03-09 23:52:41.353866: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-09 23:52:41.354521: I tensorflow/core/

Shape of training set of frames:(4, 50, 128, 128, 3)
Shape of training labels:(4, 13)
Shape of validation set of frames:(4, 50, 128, 128, 3)
Shape of validation labels:(4, 13)


2023-03-09 23:52:42.405034: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
from tensorflow.keras.layers import Input, TimeDistributed, Dense
model = models.Sequential()
model.add(layers.Conv3D(32, (1, 3, 3), activation='relu', input_shape=(50, 128, 128, 3)))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(64, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(128, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(256, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(512, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.MaxPooling3D((1, 2, 2)))

model.add(layers.Conv3D(1024, (1, 2, 2), activation='relu'))
model.add(layers.TimeDistributed(layers.BatchNormalization()))
model.add(layers.GlobalAveragePooling3D())

model.add(layers.Dense(13, activation='softmax'))


model.compile(optimizer=optimizers.Adam(CFG['LEARNING_RATE']),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 50, 126, 126, 32)  896       
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 126, 126, 32)  128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 50, 63, 63, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 50, 62, 62, 64)    8256      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 62, 62, 64)    256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 31, 31, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 30, 30, 128)   3

In [7]:
es = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)
history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[es],
                    class_weight={i:weights[i] for i in range(len(weights))})

Epoch 1/50


2023-03-09 23:55:05.950048: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 406425600 exceeds 10% of free system memory.
2023-03-09 23:55:06.085843: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 406425600 exceeds 10% of free system memory.
2023-03-09 23:55:08.661660: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 406425600 exceeds 10% of free system memory.
2023-03-09 23:55:10.412706: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 406425600 exceeds 10% of free system memory.


      1/Unknown - 6s 6s/step - loss: 0.8404 - accuracy: 0.0000e+00

2023-03-09 23:55:10.781798: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 406425600 exceeds 10% of free system memory.


    105/Unknown - 492s 5s/step - loss: 3.4573 - accuracy: 0.0762

KeyboardInterrupt: 